In [1]:
import re
import numpy as np
from scipy.spatial import distance

Python позволяет делать всякое и в пару строк, но это не по "Zen of Python"

"С кодом в первую очередь не интерпретатор работает, а человек" (с)yithian

вот так не надо:
```python
with open('sentences.txt', 'r+t', encoding='UTF-8') as f: #контекстный менеджер сам закроет файл при 'with'
    data_in = list(list(filter(None, re.split('[^a-z]', l.lower()))) for l in f)
```

In [2]:
try:
    f = open('sentences.txt', 'r+t', encoding='UTF-8')
    data_in = f.readlines()
finally:
    f.close()
    
for i in range(len(data_in)):
    data_in[i] = list(filter(None, re.split('[^a-z]', data_in[i].lower())))

In [3]:
data_in

[['in',
  'comparison',
  'to',
  'dogs',
  'cats',
  'have',
  'not',
  'undergone',
  'major',
  'changes',
  'during',
  'the',
  'domestication',
  'process'],
 ['as',
  'cat',
  'simply',
  'catenates',
  'streams',
  'of',
  'bytes',
  'it',
  'can',
  'be',
  'also',
  'used',
  'to',
  'concatenate',
  'binary',
  'files',
  'where',
  'it',
  'will',
  'just',
  'concatenate',
  'sequence',
  'of',
  'bytes'],
 ['a',
  'common',
  'interactive',
  'use',
  'of',
  'cat',
  'for',
  'a',
  'single',
  'file',
  'is',
  'to',
  'output',
  'the',
  'content',
  'of',
  'a',
  'file',
  'to',
  'standard',
  'output'],
 ['cats',
  'can',
  'hear',
  'sounds',
  'too',
  'faint',
  'or',
  'too',
  'high',
  'in',
  'frequency',
  'for',
  'human',
  'ears',
  'such',
  'as',
  'those',
  'made',
  'by',
  'mice',
  'and',
  'other',
  'small',
  'animals'],
 ['in',
  'one',
  'people',
  'deliberately',
  'tamed',
  'cats',
  'in',
  'a',
  'process',
  'of',
  'artificial',
  's

In [58]:
words_indxs = dict()
for i in range(len(data_in)):
    for j in range(len(data_in[i])):
        words_indxs.setdefault(data_in[i][j], len(words_indxs))

а это на случай проверки правильности проиндексированности: сортировка по value
```python
{k: v for k, v in sorted(words_indxs.items(), key=lambda item: item[1])}
```

In [71]:
#sentnc_words_freqs = [[0]*len(words_indxs) for i in range(len(data_in))]
#sentnc_words_freqs = np.array([[0]*len(words_indxs)] * len(data_in))
sentnc_words_freqs = np.zeros((len(data_in),len(words_indxs)), int)
for i in range(len(data_in)):
    for w in range(len(data_in[i])):
        sentnc_words_freqs[i][words_indxs[data_in[i][w]]] += 1

In [109]:
sentnc_words_freqs

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 1]])

In [110]:
cos_dist = np.zeros(len(sentnc_words_freqs), float)
for i in range(0, len(sentnc_words_freqs)):
    cos_dist[i] = distance.cosine(sentnc_words_freqs[0], sentnc_words_freqs[i])

In [119]:
cos_dist

array([0.        , 0.95275444, 0.86447381, 0.89517152, 0.77708871,
       0.94023857, 0.73273876, 0.92587507, 0.88427249, 0.90550888,
       0.83281654, 0.88047714, 0.83964325, 0.87035926, 0.87401184,
       0.94427218, 0.84063619, 0.9566445 , 0.94427218, 0.88854436,
       0.84275727, 0.82503645])

In [120]:
sentences_indxs_nearest = dict()
for i in range(len(cos_dist)):
    sentences_indxs_nearest[i] = cos_dist[i]

In [121]:
sentences_indxs_nearest

{0: 0.0,
 1: 0.9527544408738466,
 2: 0.8644738145642124,
 3: 0.8951715163278082,
 4: 0.7770887149698589,
 5: 0.9402385695332803,
 6: 0.7327387580875756,
 7: 0.9258750683338899,
 8: 0.8842724875284311,
 9: 0.9055088817476932,
 10: 0.8328165362273942,
 11: 0.8804771390665607,
 12: 0.8396432548525454,
 13: 0.8703592552895671,
 14: 0.8740118423302576,
 15: 0.9442721787424647,
 16: 0.8406361854220809,
 17: 0.956644501523794,
 18: 0.9442721787424647,
 19: 0.8885443574849294,
 20: 0.8427572744917122,
 21: 0.8250364469440588}

In [128]:
#sorted_sentences_dist = [(i,d) for i,d in sorted(sentences_indxs_nearest.items(), key=lambda item: item[1])]
sorted_sentences_dist = sorted(sentences_indxs_nearest.items(), key=lambda s: s[1])

In [129]:
sorted_sentences_dist

[(0, 0.0),
 (6, 0.7327387580875756),
 (4, 0.7770887149698589),
 (21, 0.8250364469440588),
 (10, 0.8328165362273942),
 (12, 0.8396432548525454),
 (16, 0.8406361854220809),
 (20, 0.8427572744917122),
 (2, 0.8644738145642124),
 (13, 0.8703592552895671),
 (14, 0.8740118423302576),
 (11, 0.8804771390665607),
 (8, 0.8842724875284311),
 (19, 0.8885443574849294),
 (3, 0.8951715163278082),
 (9, 0.9055088817476932),
 (7, 0.9258750683338899),
 (5, 0.9402385695332803),
 (15, 0.9442721787424647),
 (18, 0.9442721787424647),
 (1, 0.9527544408738466),
 (17, 0.956644501523794)]

In [153]:
print('1) '+' '.join(data_in[sorted_sentences_dist[0][0]])+'\n'+'2) '+' '.join(data_in[sorted_sentences_dist[1][0]])+'\n'+'3) '+' '.join(data_in[sorted_sentences_dist[2][0]])+'\n'+'XXX) '+' '.join(data_in[sorted_sentences_dist[21][0]]))

1) in comparison to dogs cats have not undergone major changes during the domestication process
2) domestic cats are similar in size to the other members of the genus felis typically weighing between and kg and lb
3) in one people deliberately tamed cats in a process of artificial selection as they were useful predators of vermin
XXX) os x mountain lion was released on july for purchase and download through apple s mac app store as part of a switch to releasing os x versions online and every year


In [155]:
try:
    f = open('submission-1.txt', 'w+t', encoding='UTF-8')
    f.write(str(sorted_sentences_dist[1][0])+' '+\
           str(sorted_sentences_dist[2][0]))
finally:
    f.close()